# qsm-chicago_model_A.ipynb
Author: Connacher Murphy

### 0. Libraries

In [61]:
import numpy as np
import pandas as pd

### 1. Calibration
We create the commuting flows matrix $\mathbf{\pi}$, where $\pi_{i n}$ denotes the share of workers living in neighborhood $i$ commuting to neighborhood $n$.

In [62]:
# Read commuting flows from csv
flows_pd = pd.read_csv('../data/chi_flows_nbhd_matrix.csv')

# Convert to numpy array
flows = flows_pd.values[:, 1:]

K = flows.shape[0] # number of neighborhoods

# Calculate population by workplace and residence
H = np.sum(flows, axis = 1, keepdims=True) # residence
L = np.sum(flows, axis = 0, keepdims=True) # workplace

# Normalize by residential population to get conditional commuting probabilities
pi = flows / H

In [63]:
theta = 3.0
alpha = 0.6

### 2. Counterfactual solver

In [67]:
# Define component functions for counterfactual solver
def cal_Z(w_tilde, theta, alpha, pi_init, A_hat):
    """
    Calculate calligraphic Z for candidate wage changes
    Arguments:
        w_tilde: candidate wage changes

        theta: preference shock dispersion parameter
        alpha: elasticity of output with respect to labor
        pi_init: initial commuting probabilities
        
        A_hat:
    """
    demand_term = np.power(
        alpha * A_hat / w_tilde,
        1 / (1 - alpha)
    )

    print(demand_term.shape)

In [65]:
# Unit test
w_tilde = np.ones(K).reshape(K, 1)

A_hat = np.ones(K).reshape(K, 1)

cal_Z(w_tilde, theta, alpha, pi, A_hat)

(77, 1)
